# OCI Generative AI 日本語化 デモ

OCI GenAIに日本語でぶん投げたプロンプトを、日本語で返す

## configuration

In [ ]:
## Install Packages
%pip install oci python-dotenv

In [ ]:
# ローカルの設定ファイルを読み込む
%load_ext dotenv
%dotenv

In [ ]:
import oci
import os

In [ ]:
# Configuration
compartment_id = os.environ.get("COMPARTMENT_ID")
config = oci.config.from_file('~/.oci/config', "DEFAULT")
endpoint = os.environ.get("GENAI_ENDPOINT")

key="key"
language_en = "en"
language_ja = "ja"

## Set Prompt

In [ ]:
# set prompt
prompt = "Oracle Cloud Infrastractureの紹介についてのブログ記事を書きたい。小学生向けのわかりやすさで、目次を考えてください。"

# set parameters
max_tokens = 600
temperature = 1.0

### exec generative ai

In [ ]:
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))
generate_text_detail = oci.generative_ai_inference.models.GenerateTextDetails()
llm_inference_request = oci.generative_ai_inference.models.CohereLlmInferenceRequest()
llm_inference_request.prompt = prompt
llm_inference_request.max_tokens = max_tokens
llm_inference_request.temperature = temperature
llm_inference_request.frequency_penalty = 0
llm_inference_request.top_p = 0.75

generate_text_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id=os.environ.get("MODEL_ID"))
generate_text_detail.inference_request = llm_inference_request
generate_text_detail.compartment_id = compartment_id
generate_text_response = generative_ai_inference_client.generate_text(generate_text_detail)

# Print result
# print(
#     generate_text_response.data.inference_response.generated_texts[0].text
# )

### exec language translation


In [ ]:
ai_language_client = oci.ai_language.AIServiceLanguageClient(config)

#text="Hello, world!"
text=generate_text_response.data.inference_response.generated_texts[0].text

In [ ]:
# Send the request to service, some parameters are not required, see API
# doc for more info
batch_language_translation_response = ai_language_client.batch_language_translation(
    batch_language_translation_details=oci.ai_language.models.BatchLanguageTranslationDetails(
        documents=[
            oci.ai_language.models.TextDocument(
                key=key,
                text=text,
                language_code=language_en)],
        compartment_id=compartment_id,
        target_language_code=language_ja),
    #opc_request_id="UUD9T8HFQPSFIX6KYQDU<unique_ID>"
    )

# Get the data from response
response_str = batch_language_translation_response.data.documents[0].translated_text

# 行頭の不要な空白を削除、改行を追加
response_str = response_str.lstrip()
response_str = response_str.replace("。", "。\n")

## Output

In [ ]:
# Print the response
print(response_str)